## Install river and scikit-learn

In [41]:
pip install "./river/"

Processing ./river
  Created wheel for river: filename=river-1-cp39-cp39-macosx_10_9_x86_64.whl size=1195855 sha256=0c8a09779057bbafa33e90de91481d1278a41217290b4388a9d61cb884e6dca2
  Stored in directory: /private/var/folders/s0/cs0fw3px6tx5srp431pdvjc80000gn/T/pip-ephem-wheel-cache-zut_xzcu/wheels/d1/6b/a3/83d8a39007debc0733461c491a8263c5af566254c5860a1a1a
Successfully built river
  Attempting uninstall: river
    Found existing installation: river 1
    Uninstalling river-1:
      Successfully uninstalled river-1
You should consider upgrading via the '/Users/brianburns/ml/my_env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "./scikit-garden/"

Processing ./scikit-garden
  Created wheel for scikit-garden: filename=scikit_garden-0.1.3-cp39-cp39-macosx_10_9_x86_64.whl size=530087 sha256=20fe418083ddcd27e2fd796fb87a854e33f904287695b8362e70b74fd4741980
  Stored in directory: /Users/brianburns/Library/Caches/pip/wheels/cc/96/10/f1f98c7ebcbb916ea7b0a696a32ae8d73b3ed876c5d2911053
Successfully built scikit-garden
  Attempting uninstall: scikit-garden
    Found existing installation: scikit-garden 0.1.3
    Uninstalling scikit-garden-0.1.3:
      Successfully uninstalled scikit-garden-0.1.3
You should consider upgrading via the '/Users/brianburns/ml/my_env/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Imports

In [1]:
from river import stream
from river.ensemble import AdaptiveRandomForestRegressorCP, AdaptiveRandomForestRegressorQRF
from skgarden.mondrian import RiverMondrianForestRegressor
import datetime
import os
import pickle
from datetime import datetime
import time
import math
import numpy as np
from river import synth
import copy

## Experiment functions

In [2]:
def get_target_variable(file_path):
    """
    Our data files have different names for the target variable.
    This returns the string name of the target variable of a dataset in an arff file.
    """
    data_stream = stream.iter_arff(file_path)
    v = next(iter(data_stream))
    return list(v[0].keys())[-1]

In [3]:
def load_datastream(arff_file):
    target_name = get_target_variable(arff_file)
    datastream = stream.iter_arff(arff_file, target = target_name)
    return datastream

In [4]:
def run_experiment(datastream, dataset_name, models_with_names, metrics_with_names, alpha, parameter_info):
    """
    datastream is a river stream object, not the path of an arff file.
    """
    # Ensure iteration doesn't exhaust datastream
    datastream = list(datastream)
    
    all_results = dict()
    all_performances = dict()
        
    for model, model_name in models_with_names:
        results = []
        performances = dict()
        time_start = time.process_time()
        i = 0
        for x,y in datastream:
            i += 1
            interval = model.predict_interval(x, alpha)
            y_hat = model.predict_one(x)
            results.append([x,y,y_hat, interval, alpha])
            model.learn_one(x,y)
            if i % 10000 == 0:
                print(model_name, dataset_name, i)

        time_end = time.process_time()
        time_elapsed = time_end - time_start
        
        for metric, metric_name in metrics_with_names:
            performances[metric_name] = metric(results)
        performances["time"] = time_elapsed
        
        all_results[model_name] = results
        all_performances[model_name] = performances

    # Save results and performances
    experiment_id = str(datetime.now()).replace(":","-").replace(".","-").replace(" ", "-")
    save_experiment(dataset_name, all_results, all_performances, alpha, experiment_id, parameter_info)

    return all_results, all_performances

In [5]:
def save_experiment(dataset_name, results, performances, alpha, experiment_id, parameter_info):
    for model_name in results.keys():
        filepath = "./results/" + dataset_name + "/" + model_name + "/" + experiment_id + "/"
        # Create directory for experiment, if it doesn't already exist
        os.makedirs(filepath, exist_ok=True)
        # save the result
        result = results[model_name]
        with open(filepath + "results.pckl", "wb") as pickle_file:
            pickle.dump(results, pickle_file)
        # save the metrics
        perf = performances[model_name]
        with open(filepath + "metrics.pckl", "wb") as pickle_file:
            pickle.dump(perf, pickle_file)
        with open(filepath + "parameters.txt","w") as params_file:
            params_file.writelines(parameter_info)

## Metrics

In [6]:
# results is a list with elements of the form [x, y, y_hat, interval, alpha]
def remove_inf_results(results):
    filtered_results = filter(lambda x: x[3][0] != -math.inf and x[3][1] != math.inf, results)
    return list(filtered_results)
    
def mean_error_rate(results):
    filtered_results = remove_inf_results(results)
    # sum instances where y isn't in confidence interval
    s = sum([x[1] < x[3][0] or x[1]> x[3][1] for x in filtered_results])
    n = len(filtered_results)
    return s/n
    

def relative_interval_size(results):
    filtered_results = remove_inf_results(results)
    y_vals = [item[1] for item in filtered_results]
    rho = max(y_vals) - min(y_vals)
    # sum length of intervals
    s = sum(x[3][1] - x[3][0] for x in filtered_results)
    n = len(filtered_results)
    return s/(rho*n)

    
def quantile_loss(results):
    filtered_results = remove_inf_results(results)
    alpha = results[0][4]
    a = alpha*relative_interval_size(results)
    def single_interval_loss(y, interval):
        return max(min(interval)-y, y- max(interval), 0)
    s = sum([single_interval_loss(x[1], x[3]) for x in filtered_results])
    y_vals = [item[1] for item in filtered_results]
    rho = max(y_vals) - min(y_vals)
    n = len(filtered_results)
    return a + (s/(n*rho))
         

def utility(results):
    filtered_results = remove_inf_results(results)
    alpha = results[0][4]
    gamma = 2*np.log(2)/alpha
    mer = mean_error_rate(results)
    ris = relative_interval_size(results)
    if mer <= alpha:
        return 1-ris
    else:
        return (1-ris)*np.exp(-gamma*(mer - alpha))

## Small-dataset experiments

In [7]:
experiment_parameters = """Experiment parameters

All river models
    - n_models = 10
    - max_features = "sqrt"
    - aggregation_method = "mean"
    - lambda_value = 1
    - leaf_prediction = "mean"

OnlineCP
    - c_max = 1000
    - update_threshold = 50 (in case of CPExact)

OnlineQRF
    - K = 200

Mondrian forests
    - n_estimators = 10
    - min_samples_split = 2
"""

In [14]:
def instantiate_models():
    models_with_names = [
        # Mondrian Forest 
        (RiverMondrianForestRegressor(n_estimators = 10, min_samples_split =2),
         "MondrianForest"),
        #CP Exact
        (AdaptiveRandomForestRegressorCP(n_models = 10, max_features = "sqrt", 
            aggregation_method = "mean", lambda_value = 1, cp_exact = True, c_max = 1000,
            update_threshold = 50, leaf_prediction = "mean"),
          "CPExact"),
#         # OnlineQRF
#         (AdaptiveRandomForestRegressorQRF(n_models = 10, max_features = "sqrt", 
#             aggregation_method = "mean", lambda_value = 1, k_sketch = 200, leaf_prediction = "mean"),
#          "OnlineQRF"),
         
#         # CP Approx
#         (AdaptiveRandomForestRegressorCP(n_models = 10, max_features = "sqrt", 
#             aggregation_method = "mean", lambda_value = 1, cp_exact = False, c_max = 1000, leaf_prediction = "mean"),
#          "CPApprox"),
    ]
    return models_with_names

In [8]:
metrics_with_names = [
        (mean_error_rate, "MER"),
        (relative_interval_size, "RIS"),
        (quantile_loss, "QL"),
        (utility, "Utility")
    ]

In [345]:
ds = list(load_datastream("data/stationary/house_16H.arff"))
dataset_name = "house_16H"
alpha = 0.1
models_with_names = instantiate_models()

In [346]:
res = run_experiment(ds[0:5000], dataset_name, models_with_names, metrics_with_names, alpha, experiment_parameters)

1000
2000
3000
4000
5000
0.016654410801487593 10739.95717562631 500001.0 4999 715889.2666438702
1000
2000
3000
4000
5000
0.020715605079162175 5205.023937761035 500001.0 4999 198876.54758972657
1000
2000
3000
4000
5000
0.022704047690524344 2776.5571114222844 500001.0 4999 451300.0
1000
2000
3000
4000
5000
0.02490731531714653 4312.056399734822 500001.0 4999 225160.0


In [348]:
res[1]

{'MondrianForest': {'MER': 0.27485497099419887,
  'RIS': 0.16654410801487593,
  'QL': 0.03813428219299743,
  'Utility': 0.0738160493423135,
  'time': 46.63450399999965},
 'CPExact': {'MER': 0.08261652330466093,
  'RIS': 0.20715605079162175,
  'QL': 0.031125632134630134,
  'Utility': 0.7928439492083783,
  'time': 16.13814100000036},
 'OnlineQRF': {'MER': 0.03780756151230246,
  'RIS': 0.22704047690524343,
  'QL': 0.02825715080716268,
  'Utility': 0.7729595230947566,
  'time': 11.062611999999717},
 'CPApprox': {'MER': 0.0598119623924785,
  'RIS': 0.24907315317146528,
  'QL': 0.03353141086842507,
  'Utility': 0.7509268468285347,
  'time': 19.601196999999956}}

In [9]:
def run_small_experiments(n_repeats):
    datasets = os.listdir("data/stationary/")
    datasets.remove(".DS_Store")
    for i in range(n_repeats):
        for dataset in datasets:
            print(dataset)
            models_with_names = instantiate_models()
            alpha = 0.1
            datastream = list(load_datastream("data/stationary/" + dataset))
            dataset_name = dataset.replace(".arff","")
            run_experiment(datastream, dataset_name, models_with_names, metrics_with_names, alpha, experiment_parameters)

In [11]:
run_small_experiments(10)

yprop_4_1.arff
newsPopularity.arff
energy.arff
kin8nm.arff
elevators.arff
cpu_act.arff
house_8L.arff
puma8NH.arff
fried.arff
2dplanes.arff
calHousing.arff
house_16H.arff
ailerons.arff
sulfur.arff
yprop_4_1.arff
newsPopularity.arff
energy.arff
kin8nm.arff
elevators.arff
cpu_act.arff
house_8L.arff
puma8NH.arff
fried.arff
2dplanes.arff
calHousing.arff
house_16H.arff
ailerons.arff
sulfur.arff
yprop_4_1.arff
newsPopularity.arff
energy.arff
kin8nm.arff
elevators.arff
cpu_act.arff
house_8L.arff
puma8NH.arff
fried.arff
2dplanes.arff
calHousing.arff
house_16H.arff
ailerons.arff
sulfur.arff
yprop_4_1.arff
newsPopularity.arff
energy.arff
kin8nm.arff
elevators.arff
cpu_act.arff
house_8L.arff
puma8NH.arff
fried.arff
2dplanes.arff
calHousing.arff
house_16H.arff
ailerons.arff
sulfur.arff
yprop_4_1.arff
newsPopularity.arff
energy.arff
kin8nm.arff
elevators.arff
cpu_act.arff
house_8L.arff
puma8NH.arff
fried.arff
2dplanes.arff
calHousing.arff
house_16H.arff
ailerons.arff
sulfur.arff
yprop_4_1.arff
newsP

## Concept-drift experiments

In [10]:
# Experiment with Friedman data

In [11]:
# Functions which introduce concept-drift to an underlying Friedman datastream of 10 features

In [12]:
def cd_permute(datapoint:dict):
    d = copy.deepcopy(datapoint)
    d[0] = datapoint[3]
    d[1] = datapoint[4]
    d[2] = datapoint[1]
    d[3] = datapoint[0]
    d[4] = datapoint[2]
    return d

In [13]:
def cd_dataset(dataset):
    """
    Permute the positions of the independent variables starting 25% through the dataset.
    Revert to original permutation of independent variables starting 75% through the dataset.
    """
    n = len(dataset)
    n_start = int(0.25*n)
    n_end = int(0.75*n)
    dataset_new = copy.deepcopy(dataset)
    for i in range(n_start, n_end):
        dataset_new[i] = (cd_permute(dataset_new[i][0]), dataset_new[i][1])
    return dataset_new

In [14]:
# Load the Friedman data

In [15]:
datastream = synth.Friedman(seed=28)
datastream = list(datastream.take(1000000))
cd_datastream = cd_dataset(datastream)

In [16]:
# On large concept-drift data, we evaluate metrics on tumbling windows of size 10000.
# We modify our metrics to be evaluated on arithmetically-increasing subsets of our data.

In [11]:
# Given a metric, returns function which computes that metric on arithmetically-increasing
# subsets of the data, with tumbling window = window_size.
def tumbling_metric(metric, window_size):
    def t_metric(results):
        metric_results = []
        for i in range(window_size, len(results), window_size):
            perf = metric(results[i - window_size:i])
            metric_results.append([i, perf])
        return metric_results
    return t_metric

In [18]:
dataset_name = "cd_friedman1_global_abrupt"
models_with_names = instantiate_models()
metrics_with_names = [
        (tumbling_metric(mean_error_rate, 10000), "MER"),
        (tumbling_metric(relative_interval_size, 10000), "RIS"),
        (tumbling_metric(quantile_loss, 10000), "QL"),
        (tumbling_metric(utility, 10000), "Utility")
    ]
alpha = 0.1
parameter_info = """Experiment parameters

All river models
    - n_models = 10
    - max_features = "sqrt"
    - aggregation_method = "mean"
    - lambda_value = 1
    - leaf_prediction = "mean"

OnlineCP
    - c_max = 1000
    - update_threshold = 50 (in case of CPExact)

OnlineQRF
    - K = 200

Mondrian forests
    - n_estimators = 10
    - min_samples_split = 2
"""

In [19]:
s = run_experiment(cd_datastream, dataset_name, models_with_names, metrics_with_names, alpha, parameter_info)

CPApprox cd_friedman1_global_abrupt 10000
CPApprox cd_friedman1_global_abrupt 20000
CPApprox cd_friedman1_global_abrupt 30000
CPApprox cd_friedman1_global_abrupt 40000
CPApprox cd_friedman1_global_abrupt 50000
CPApprox cd_friedman1_global_abrupt 60000
CPApprox cd_friedman1_global_abrupt 70000
CPApprox cd_friedman1_global_abrupt 80000
CPApprox cd_friedman1_global_abrupt 90000
CPApprox cd_friedman1_global_abrupt 100000
CPApprox cd_friedman1_global_abrupt 110000
CPApprox cd_friedman1_global_abrupt 120000
CPApprox cd_friedman1_global_abrupt 130000
CPApprox cd_friedman1_global_abrupt 140000
CPApprox cd_friedman1_global_abrupt 150000
CPApprox cd_friedman1_global_abrupt 160000
CPApprox cd_friedman1_global_abrupt 170000
CPApprox cd_friedman1_global_abrupt 180000
CPApprox cd_friedman1_global_abrupt 190000
CPApprox cd_friedman1_global_abrupt 200000
CPApprox cd_friedman1_global_abrupt 210000
CPApprox cd_friedman1_global_abrupt 220000
CPApprox cd_friedman1_global_abrupt 230000
CPApprox cd_friedman

OnlineQRF cd_friedman1_global_abrupt 900000
OnlineQRF cd_friedman1_global_abrupt 910000
OnlineQRF cd_friedman1_global_abrupt 920000
OnlineQRF cd_friedman1_global_abrupt 930000
OnlineQRF cd_friedman1_global_abrupt 940000
OnlineQRF cd_friedman1_global_abrupt 950000
OnlineQRF cd_friedman1_global_abrupt 960000
OnlineQRF cd_friedman1_global_abrupt 970000
OnlineQRF cd_friedman1_global_abrupt 980000
OnlineQRF cd_friedman1_global_abrupt 990000
OnlineQRF cd_friedman1_global_abrupt 1000000
CPExact cd_friedman1_global_abrupt 10000
CPExact cd_friedman1_global_abrupt 20000
CPExact cd_friedman1_global_abrupt 30000
CPExact cd_friedman1_global_abrupt 40000
CPExact cd_friedman1_global_abrupt 50000
CPExact cd_friedman1_global_abrupt 60000
CPExact cd_friedman1_global_abrupt 70000
CPExact cd_friedman1_global_abrupt 80000
CPExact cd_friedman1_global_abrupt 90000
CPExact cd_friedman1_global_abrupt 100000
CPExact cd_friedman1_global_abrupt 110000
CPExact cd_friedman1_global_abrupt 120000
CPExact cd_friedman1

MondrianForest cd_friedman1_global_abrupt 730000
MondrianForest cd_friedman1_global_abrupt 740000
MondrianForest cd_friedman1_global_abrupt 750000
MondrianForest cd_friedman1_global_abrupt 760000
MondrianForest cd_friedman1_global_abrupt 770000
MondrianForest cd_friedman1_global_abrupt 780000
MondrianForest cd_friedman1_global_abrupt 790000
MondrianForest cd_friedman1_global_abrupt 800000
MondrianForest cd_friedman1_global_abrupt 810000
MondrianForest cd_friedman1_global_abrupt 820000
MondrianForest cd_friedman1_global_abrupt 830000
MondrianForest cd_friedman1_global_abrupt 840000
MondrianForest cd_friedman1_global_abrupt 850000
MondrianForest cd_friedman1_global_abrupt 860000
MondrianForest cd_friedman1_global_abrupt 870000
MondrianForest cd_friedman1_global_abrupt 880000
MondrianForest cd_friedman1_global_abrupt 890000
MondrianForest cd_friedman1_global_abrupt 900000
MondrianForest cd_friedman1_global_abrupt 910000
MondrianForest cd_friedman1_global_abrupt 920000
MondrianForest cd_fr

In [110]:
#CPAprrox takes 1 minute to go through  10k datapoints., at 420k datapoints.
#Takes 1 minute 50 seconds to go thrhough 10k datapoints at 750k datapoints!

({0: 0.016410949070087333,
  1: 0.2782558469335449,
  2: 0.8185684229585304,
  3: 0.7683679899654309,
  4: 0.2148203408429732,
  5: 0.984567363665633,
  6: 0.5287647187545143,
  7: 0.7676221057186963,
  8: 0.4693229444473781,
  9: 0.28530809435445836},
 13.320901360071973)

In [111]:
datastream[300000]

({0: 0.7683679899654309,
  1: 0.8185684229585304,
  2: 0.2148203408429732,
  3: 0.016410949070087333,
  4: 0.2782558469335449,
  5: 0.984567363665633,
  6: 0.5287647187545143,
  7: 0.7676221057186963,
  8: 0.4693229444473781,
  9: 0.28530809435445836},
 13.320901360071973)

# Airlines dataset (700k)

In [9]:
airlines_ds = list(load_datastream("data/airlines/plane_700K_train.arff"))

In [15]:
dataset_name = "cd_airlines_700k"
models_with_names = instantiate_models()
metrics_with_names = [
        (tumbling_metric(mean_error_rate, 10000), "MER"),
        (tumbling_metric(relative_interval_size, 10000), "RIS"),
        (tumbling_metric(quantile_loss, 10000), "QL"),
        (tumbling_metric(utility, 10000), "Utility")
    ]
alpha = 0.1
parameter_info = """Experiment parameters

All river models
    - n_models = 10
    - max_features = "sqrt"
    - aggregation_method = "mean"
    - lambda_value = 1

OnlineCP
    - c_max = 1000
    - update_threshold = 50 (in case of CPExact)

OnlineQRF
    - K = 200

Mondrian forests
    - n_estimators = 10
    - min_samples_split = 2
"""

In [16]:
s = run_experiment(airlines_ds, dataset_name, models_with_names, metrics_with_names, alpha, parameter_info)

MondrianForest cd_airlines_700k 10000
MondrianForest cd_airlines_700k 20000
MondrianForest cd_airlines_700k 30000
MondrianForest cd_airlines_700k 40000
MondrianForest cd_airlines_700k 50000
MondrianForest cd_airlines_700k 60000
MondrianForest cd_airlines_700k 70000
MondrianForest cd_airlines_700k 80000
MondrianForest cd_airlines_700k 90000
MondrianForest cd_airlines_700k 100000
MondrianForest cd_airlines_700k 110000
MondrianForest cd_airlines_700k 120000
MondrianForest cd_airlines_700k 130000
MondrianForest cd_airlines_700k 140000
MondrianForest cd_airlines_700k 150000
MondrianForest cd_airlines_700k 160000
MondrianForest cd_airlines_700k 170000
MondrianForest cd_airlines_700k 180000
MondrianForest cd_airlines_700k 190000
MondrianForest cd_airlines_700k 200000
MondrianForest cd_airlines_700k 210000
MondrianForest cd_airlines_700k 220000
MondrianForest cd_airlines_700k 230000
MondrianForest cd_airlines_700k 240000
MondrianForest cd_airlines_700k 250000
MondrianForest cd_airlines_700k 26